In [1]:
import json
import pandas as pd
import numpy as np
import nibabel as nb

In [2]:
table = pd.read_table('../cerebellify/rawmaps/about_maps.txt',
                      skiprows=38, nrows=28, delim_whitespace=True, 
                      header=None, names=('key', 'id', 'Number'))
table['parent'] = table.id.apply(lambda x: 'Lobule_' + x.split('_')[1])
table

,key,id,Number,parent
0,1,Left_I-IV,1001,Lobule_I-IV
1,2,Right_I-IV,1001,Lobule_I-IV
2,3,Left_V,2002,Lobule_V
3,4,Right_V,2002,Lobule_V
4,5,Left_VI,2101,Lobule_VI
5,6,Vermis_VI,2101,Lobule_VI
6,7,Right_VI,2101,Lobule_VI
7,8,Left_CrusI,2102,Lobule_CrusI
8,9,Vermis_CrusI,2102,Lobule_CrusI
9,10,Right_CrusI,2102,Lobule_CrusI


In [3]:
info =  {'header': {'citation': '',
                    'contact': 'satra@mit.edu',
                    'description': 'HAWG version of a single human cerebellar atlas',
                    'license': 'PDDL',
                    'name': 'Cerebellum Human 1',
                    'provenance': 'Some text or something more elaborate?',
                    'species': 'Homo Sapiens',
                    'version': '1.0.0'},
         'ofkat': {},
         'resources': {'labelsrc': {'format': 'nifti',
                                    'location': 'chuman_1.nii',
                                    'name': 'discrete labels of cortical regions',
                                    'type': 'discrete'}}}

In [4]:
def get_coord(imgfile, id):
    data = nb.load(imgfile).get_data()
    return np.array([val[0] for val in np.nonzero(data == id)]).tolist()

In [5]:
regions = {}
# include each unique label
for item in table.iterrows():
    regions[item[1].id] = {"description": "Jason: {}".format(item[1].id),
                           "display_name": item[1].id.replace('_', ' '),
                           "part_of": [item[1].parent],
                           "resources": {
                               "labelsrc": {
                                   "example_color": np.random.randint(0, 256, size=(3,)).tolist(),
                                   "example_coord": list(get_coord('../cerebellify/rawmaps/chuman_1.nii', item[1].key)),
                                   "key": item[1].key
                               }
                           }
                          }
# include each lobule as a parent group
for val in table.groupby('parent'):
    sub_regions = val[1].id.values.tolist()
    regions[val[0]] = {"description": "Jason: {}".format(val[0]),
                       "display_name": val[0].replace('_', ' '),
                       "part_of": ['Cerebellum'],
                       "resources": {
                           "example_color": np.random.randint(0, 256, size=(3,)).tolist(),
                       },
                       "includes": sub_regions
                      }

info['ofkat'] = regions

In [7]:
with open('cerebellum-single.json', 'wt') as fp:
    json.dump(info, fp, indent=2, sort_keys=True)